# 기초 MLP 실습

import torch

In [ ]:
import torch
import torch.nn as nn

device check(gpu)

In [ ]:
# device check

## XOR problem

![img](https://github.com/june-oh/2023_AI_Academy_ASR/blob/main/img/xor_table.jpg?raw=true)

data, label

In [ ]:
# define X, Y


In [ ]:
# define torch.tensor X, Y


In [ ]:
# check X, Y type

## perceptron

<img src="https://github.com/june-oh/2023_AI_Academy_ASR/blob/main/img/perceptron.jpg?raw=true" width=450>

nn.sequential

nn.linear(arg)

- arg 
  - n_in_node : 입력 노드의 개수
  - n_out_node : 출력 노드의 개수



In [ ]:
??nn.Linear

In [ ]:
??nn.Sigmoid

In [ ]:
# define model


In [ ]:
# check model 


In [ ]:
??torch.nn.BCELoss

In [ ]:
??torch.optim.SGD

In [ ]:
# model parameter check

In [ ]:
# define loss_fn , optimizer


### Train loop

- epoch (전체데이터)
  - batch (batch data)


In [ ]:
# Train 

In [ ]:
# check loss


In [ ]:
# Inference

## define MLP model

In [ ]:
# define mlp model

In [ ]:
# model parameter check

모델을 학습할때 lr을 충분히 크게(e.g., lr = 0.1) 줘야 학습 가능

In [ ]:
#Train

In [ ]:
# check loss

In [ ]:
# final loss